# Install+Import

In [1]:
import sys 
from sklearn.model_selection import KFold

sys.path.append('/kaggle/input/rsna-2023-abdominal-trauma')
sys.path.append('/kaggle/input/atd-packages/mypackages')

In [2]:
import numpy as np
import pandas as pd
from glob import glob
import torch
import torch.nn as nn
from timeit import default_timer as timer
from utils import get_seg_model,load_dicom_folder,resize3d,visualize,get_crops
from utils import LSKModel,BowelModel,PENet
from utils import score,create_training_solution
from matplotlib import pyplot as plt
import gc
import os
    
print('IMPORT OK!!!!')

IMPORT OK!!!!


In [3]:
mode = 'local' #submit #local
models_dir="/kaggle/input/rsna-2023-abdominal-trauma"
if mode =='local':
    image_dir  = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'
  
    df_meta = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')
    df_p = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv') 
    kf = KFold(5,shuffle=True,random_state=42)
    df_p["fold"]=-1
    for fold, (train_idx, valid_idx) in enumerate(kf.split(df_p,df_p)):
        df_p.loc[valid_idx, 'fold'] = fold

    df_p=df_p[df_p["fold"]==0]    
    df_p=df_p.sort_values(by="patient_id").reset_index(drop=True)  
    df_meta= df_meta.merge(df_p["patient_id"],on="patient_id")

else:
    image_dir = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images'
    
    df_meta = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/test_series_meta.csv')
  

df_meta.sort_values(by=["patient_id","aortic_hu"],inplace=True)
df_meta.set_index(["patient_id","series_id"],inplace=True)

# Models

In [4]:
if mode=="local":
    select_models=[0]
else:
    select_models=[0,1,2,3,4]

def load_seg_models(models_dir,kernel):
    models=[]
    for ifold in range(5):
        model=get_seg_model(pretrained=False)
        model.load_state_dict(torch.load(f"{models_dir}/{kernel}_fold{ifold}_best.pth"))
        models.append(model)
    return models

def load_lsk_models(models_dir,kernel,backbone):
    models=[]
    for ifold in select_models:
        model=LSKModel(backbone,pretrained=False)
        model.load_state_dict(torch.load(f"{models_dir}/{kernel}_fold{ifold}_best.pth"))
        models.append(model)
    return models


def load_bowel_models(models_dir,kernel,backbone):
    models=[]
    for ifold in select_models:
        model=BowelModel(backbone,pretrained=False)
        model.load_state_dict(torch.load(f"{models_dir}/{kernel}_fold{ifold}_best.pth"))
        models.append(model)
    return models

def load_final_models(models_dir):
    models=[]
    for ifold in select_models:
        model=PENet()
        model.load_state_dict(torch.load(f"{models_dir}/lstm _fold{ifold}_best.pth"))
        models.append(model)
    return models


models_seg_dir=os.path.join(models_dir,"models_seg")
kernel_seg = 'timm3d_res18d_unet4b_128_128_128_dsv2_flip12_shift333p7_gd1p5_bs4_lr3e4_20x50ep'
seg_models=load_seg_models(models_seg_dir,kernel_seg)

models_lsk_dir=os.path.join(models_dir,"models_cls")
kernel_lsk = '0920_1bonev2_effv2s_224_15_6ch_augv2_mixupp5_drl3_rov1p2_bs8_lr23e5_eta23e6_50ep'
backbone_lsk1="tf_efficientnetv2_b2.in1k"
lsk_models=load_lsk_models(models_lsk_dir,kernel_lsk,backbone_lsk1)

models_bowel_dir=os.path.join(models_dir,"models_bowel")
kernel_bowel = '0920_1bonev2_effv2s_224_15_6ch_augv2_mixupp5_drl3_rov1p2_bs8_lr23e5_eta23e6_50ep'
backbone_bowel1="tf_efficientnetv2_b0.in1k"
bowel_models=load_bowel_models(models_bowel_dir,kernel_bowel,backbone_bowel1)

models_final_dir=os.path.join(models_dir,"models_stage3_all")
final_models=load_final_models(models_final_dir)



In [5]:
image_seg_size = [128, 128, 128]
#image_check_dir="./preprocessed1"
def resize_image(image_full,sorted_slices,series_id):    
    image1=resize3d(image_full,sorted_slices,image_seg_size)
    #image_check_file=os.path.join(image_check_dir,f"{series_id}_mask.npy")
    #image_check=np.load(image_check_file)
    #image_diff=image1-image_check
    #df_describe=pd.DataFrame(image_diff.flatten()[:,None])
    #print(df_describe.describe())
    #print("image diff",np.mean(np.abs(image_diff)))
    image=(image1/255.).astype(np.float32)
    return image

In [6]:

def infer_mask(image,models,vis=False):
    with torch.cuda.amp.autocast(enabled=True):
        with torch.no_grad():
            image=torch.from_numpy(image).cuda()
            image=image[None,None,...]
            pred_masks = []
            for model in models:
                model.cuda()
                model.eval()
                pmask = model(image).squeeze()
                pmask=nn.Softmax(dim=0)(pmask)
                pred_masks.append(pmask)
    mask = torch.stack(pred_masks).mean(0)
    mask=torch.argmax(mask,dim=0)
    mask=mask.numpy(force=True).astype(np.uint8)
  
    return mask

In [7]:
def infer_lsk(image_crops,models,nslice=24):
    lsk_dict={0:"liver",1:"spleen",2:"kidney"}
    results={}
    with torch.cuda.amp.autocast(enabled=True):
        with torch.no_grad():
            for iorgan in range(3): #liver,spleen,kidney
                inds=list(range(iorgan*nslice,(iorgan+1)*nslice))
                images=image_crops[inds,...]
                images=torch.from_numpy(images).float().cuda()
                images=images[None,...]
                mlogits=[]
                for model in models:
                    model.cuda()
                    model.eval()
                    logits = model(images).mean(dim=1).squeeze()
                    logits = logits.numpy(force=True)
                    mlogits.append(logits)
                    #print(logits)
                results[lsk_dict[iorgan]]=np.mean(np.stack(mlogits),axis=0)
    return results
       

In [8]:
def infer_be(image_crops,models,start=3*24,nslice=128):
    results={}
    with torch.cuda.amp.autocast(enabled=True):
        with torch.no_grad():
            inds=list(range(start,start+nslice))
            images=image_crops[inds,...]
            images=torch.from_numpy(images).cuda()
            images=images[None,...]
            mlogits=[]
            for model in models:
                model.cuda()
                model.eval()
                logits = model(images)[1].squeeze()
                logits = logits.numpy(force=True)
                mlogits.append(logits)
            mlogits=np.mean(np.stack(mlogits),axis=0)
            results["bowel"]=mlogits
            #print(mlogits)
    return results

In [9]:
def infer_final(data,models):
    results={"liver":[],"spleen":[],"kidney":[],"bowel":[],"extravasation":[]}
    probs={}
    with torch.cuda.amp.autocast(enabled=True):
        with torch.no_grad():
            for k in data:
                data[k]=torch.from_numpy(data[k]).float().cuda()[None,...]
            for model in models:
                model.cuda()
                model.eval()
                logits = model(data)
                for k in logits:
                    results[k].append(logits[k].squeeze())
    for k in results:
        results[k]=torch.mean(torch.stack(results[k]),axis=0)
        if k in ["liver","spleen","kidney"]:
            probs[k]=nn.Softmax(dim=0)(results[k].float())
        else:
            probs[k]=nn.Sigmoid()(results[k].float())
        probs[k]=probs[k].numpy(force=True)
    return probs

In [10]:
def visualize_lsk(cropped_images):
    f, axarr = plt.subplots(2,4)
    p=0
    for organ in range(3):
        imgs=cropped_images[list(range(organ*24,(organ+1)*24))]
        sel=7
        axarr[0, p].imshow(255*imgs[sel][1],cmap="gray")
        axarr[1, p].imshow(imgs[sel][-1],cmap="gray")
        p+=1
    imgs=cropped_images[(3*24):]
    sel=65
    axarr[0, p].imshow(255*imgs[sel][1],cmap="gray")
    axarr[1, p].imshow(imgs[sel][-1])




In [11]:
vis=False
mask_dir="./preprocessed1"
crops_lsk_dir="./preprocessed2"
crops_bowel_dir="./bowel"
def analyze_dicom(patient_id,series_id):
    start_timer = timer()
    image_folder=f"{image_dir}/{int(patient_id)}/{int(series_id)}"
    image_full,sorted_slices=load_dicom_folder(image_folder,None)
    load_time=timer()-start_timer
    image=resize_image(image_full,sorted_slices,series_id)
    mask=infer_mask(image,seg_models,vis=vis)
    if vis:
        visualize(image,mask)
    #mask_file=os.path.join(mask_dir,f"{series_id}_mask.npy")
    #check_mask=np.load(mask_file)
    #mask_diff=mask-check_mask
    #df_describe = pd.DataFrame(mask_diff.flatten()[:,None])
    #print("mask diff",df_describe.describe())
    #visualize(mask_diff,mask_diff)

    gc.collect()
    torch.cuda.empty_cache()
    cropped_images = get_crops(image_full,mask) 
    
    if vis:
        visualize_lsk(cropped_images)
        lsk_filepath=os.path.join(crops_lsk_dir, f'{patient_id}_{series_id}.npz')
        bowel_filepath=os.path.join(crops_bowel_dir,f'{patient_id}_{series_id}.npz')
        image_check_lsk = np.load(lsk_filepath)['arr_0'] # slice,channel,H,W
        image_check_bowel = np.load(bowel_filepath)['arr_0'] # slice,channel,H,W
        image_check=np.concatenate((image_check_lsk,image_check_bowel),axis=0)
        diff=image_check-cropped_images

        visualize_lsk(diff)
        print(diff.shape)
        df_describe = pd.DataFrame(diff[:72,0:3,:,:].flatten()[:,None])
        print(df_describe.describe())

        #df_describe = pd.DataFrame(diff[72:,0:3,:,:].flatten()[:,None])
        #print(df_describe.describe())

    cropped_images=cropped_images.astype(np.float32)/255.
    lsk_logits=infer_lsk(cropped_images,lsk_models)
    gc.collect()
    torch.cuda.empty_cache()
    be_logits=infer_be(cropped_images,bowel_models)
    gc.collect()
    torch.cuda.empty_cache()
    logits=lsk_logits
    logits.update(**be_logits)
    process_time=timer()-start_timer-load_time  
    print(f"load time {load_time:.1f} process time {process_time:.1f}")
    return logits

# Run

In [12]:
def run(num=None):
    submit_df_data = []
    start_timer = timer()
    count=0
    npatients=df_meta.reset_index()['patient_id'].nunique()
    df_meta_run=df_meta.copy()
    if num is not None:
        df_meta_run=df_meta_run.iloc[:num]
    for patient_id, group in df_meta_run.groupby(level=0):
        count+=1
        plogits={"liver":[],"spleen":[],"kidney":[],"bowel":[]}
        for ind,row in group.iterrows():
            assert(len(group) in [1,2])
            patient_id,series_id=ind
            #end='\r'
            print(f"{count}/{npatients} patient {patient_id},series {series_id} time {timer() - start_timer:.1f} secs", flush=True)

         
            logits=analyze_dicom(patient_id,series_id)
            for k in plogits:
                logits[k]=np.concatenate((logits[k],np.array((row.aortic_hu,))))
                plogits[k].append(logits[k])
        if len(group)==1:
            for k in plogits:
                plogits[k].append(plogits[k][0])
        for k in plogits:
            plogits[k]=np.stack(plogits[k])
        probs=infer_final(plogits,final_models)
        submit_df_data.append({
                    'patient_id': patient_id,
                    'series_id' : series_id,
                    'liver_healthy' : probs["liver"][0],
                    'liver_low'     : probs["liver"][1],
                    'liver_high'    : probs["liver"][2],
                    'spleen_healthy': probs["spleen"][0],
                    'spleen_low'    : probs["spleen"][1],
                    'spleen_high'   : probs["spleen"][2],
                    'kidney_healthy': probs["kidney"][0],
                    'kidney_low'    : probs["kidney"][1],
                    'kidney_high'   : probs["kidney"][2],

                    'bowel_healthy'        : 1-probs["bowel"],
                    'bowel_injury'         : probs["bowel"],
                    'extravasation_healthy': 1-probs["extravasation"],
                    'extravasation_injury' : probs["extravasation"],
                })
        
    submit_col=[
            'patient_id',
            'bowel_healthy','bowel_injury','extravasation_healthy','extravasation_injury','kidney_healthy','kidney_low','kidney_high','liver_healthy','liver_low','liver_high','spleen_healthy','spleen_low','spleen_high'
        ]

    submit_df = pd.DataFrame(submit_df_data)
    submit_df = submit_df[submit_col]
    submit_df.to_csv('submission.csv',index=False)
            


In [13]:
RUN=True
if RUN:
    run()

1/630 patient 263,series 44610 time 0.0 secs
2/630 patient 403,series 9 time 5.0 secs
2/630 patient 403,series 7698 time 16.2 secs
3/630 patient 581,series 43593 time 21.7 secs
3/630 patient 581,series 33994 time 25.2 secs
4/630 patient 697,series 40638 time 28.9 secs
4/630 patient 697,series 25035 time 34.7 secs
5/630 patient 706,series 28342 time 45.8 secs
5/630 patient 706,series 53270 time 51.6 secs
6/630 patient 766,series 49801 time 55.3 secs
6/630 patient 766,series 23185 time 68.7 secs
7/630 patient 820,series 11921 time 74.1 secs
7/630 patient 820,series 38809 time 77.9 secs
8/630 patient 980,series 49378 time 82.5 secs
8/630 patient 980,series 57720 time 86.7 secs
9/630 patient 1278,series 34096 time 91.1 secs
10/630 patient 1330,series 32979 time 93.9 secs
10/630 patient 1330,series 18309 time 104.8 secs
11/630 patient 1464,series 28340 time 110.8 secs
12/630 patient 1523,series 62556 time 127.6 secs
13/630 patient 1644,series 7854 time 130.7 secs
14/630 patient 1758,series 

In [14]:
if mode=="local":
     submission=pd.read_csv("submission.csv")
     
     solution=create_training_solution(df_p)
     solution=solution.merge(submission[["patient_id"]],on="patient_id")
     valid_loss,group_losses = score(solution.copy(),submission,'patient_id')
     print("# patients",len(solution))
     print(group_losses)
     print("valid_loss",valid_loss)
     

# patients 630
{'bowel': 0.12818486835160303, 'extravasation': 0.6107858119151798, 'kidney': 0.33250891300949986, 'liver': 0.4118837722478592, 'spleen': 0.43727035469269604, 'any_injury': 0.5332865782561523}
valid_loss 0.408986716412165
